# 자유음성 데이터

변환한 CSV 파일을 이용해서 전처리하는 과정입니다.

In [83]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import re

In [84]:
dataSetPath = ''
saveSetPath = ''

splitRate = 0.1 # valid

In [85]:
df = pd.read_csv(dataSetPath)
df

,file_name,gender,age,record_time,text
0,일반남여_일반통합15_F_1551284796_28_수도권_실내_25231.wav,F,28,3.24,말되네 먹방이 그렇게 유행이더니
1,일반남여_일반통합15_F_1534603642_27_수도권_실내_25771.wav,F,27,3.92,뭐가 믿을 수가 없어 진짜 성격 이상해 가끔 보면
2,일반남여_일반통합11_F_libelian_33_기타_실내_15677.wav,F,33,8.08,네 아무래도 그렇죠 그리고 방송에서는 거르는게 너무 많잖아요
3,일반남여_일반통합07_F_1571583357_40_수도권_실내_10130.wav,F,40,2.90,일주일에서 열흘 정도
4,일반남여_일반통합07_F_1581418850_38_수도권_실내_09965.wav,F,38,3.07,왜 안 간지 꽤 됐잖아
...,...,...,...,...,...
787553,일반남여_일반통합13_M_1577813879_31_수도권_실내_19816.wav,M,31,5.72,여행 떠나는 날 나한테 전화해서 자랑한 거 기억 안 나요
787554,일반남여_일반통합13_M_1577813879_31_수도권_실내_19613.wav,M,31,5.72,다음주 토요일 박교수님 큰아들 결혼식 있는 거 알고 있지
787555,일반남여_일반통합13_M_1577813879_31_수도권_실내_19615.wav,M,31,5.12,그러게 말이야 시간이 어떻게 가는지 모르고 사는 것 같네
787556,일반남여_일반통합13_M_1577813879_31_수도권_실내_20201.wav,M,31,8.02,저도 처음에는 그런 줄 알았는데 내가 족발 이러면 화를 내더라고요 맨날 그런 것만 ...


In [87]:
def special_filter(sentence):
    '''
    문자 단위로 특수 문자 및 노이즈 표기 필터링해주는 함수이다.
    특수 문자를 아예 필터링 해버리면 문제가 되는 '#', '%'와 같은 문자를 확인하고, 문제가 되는 특수 문자는 해당 발음으로 바꿔주었다.
    '''
    SENTENCE_MARK = ['?', '!']
    NOISE = ['o', 'n', 'u', 'b', 'l']
    EXCEPT = ['/', '+', '*', '-', '@', '$', '^', '&', '[', ']', '=', ':', ';', ',']
    
    new_sentence = str()
    for idx, ch in enumerate(sentence):
        if ch not in SENTENCE_MARK:
            # o/, n/ 등 처리
            if idx + 1 < len(sentence) and ch in NOISE and sentence[idx+1] == '/': 
                continue 

        if ch == '#': 
            new_sentence += '샾'

        elif ch not in EXCEPT: 
            new_sentence += ch

    pattern = re.compile(r'\s\s+')
    new_sentence = re.sub(pattern, ' ', new_sentence.strip())
    return new_sentence

In [88]:
df['text'] = df['text'].apply(special_filter)
df['text']

0                                         말되네 먹방이 그렇게 유행이더니
1                               뭐가 믿을 수가 없어 진짜 성격 이상해 가끔 보면
2                         네 아무래도 그렇죠 그리고 방송에서는 거르는게 너무 많잖아요
3                                               일주일에서 열흘 정도
4                                              왜 안 간지 꽤 됐잖아
                                ...                        
787553                      여행 떠나는 날 나한테 전화해서 자랑한 거 기억 안 나요
787554                      다음주 토요일 박교수님 큰아들 결혼식 있는 거 알고 있지
787555                      그러게 말이야 시간이 어떻게 가는지 모르고 사는 것 같네
787556    저도 처음에는 그런 줄 알았는데 내가 족발 이러면 화를 내더라고요 맨날 그런 것만 ...
787557                                 뭘 그렇게 또 야박하게 선을 그으세요
Name: text, Length: 787558, dtype: object

In [89]:
# 이상한 wav 파일 제거

no_audio_list = ['일반남여_일반통합12_M_1571527490_28_충청_실내_17186.wav', '일반남여_일반통합13_M_zzqkfkazz_25_기타_실내_19992.wav']

no_audio_index = df[(df['file_name'] == no_audio_list[0]) | (df['file_name'] == no_audio_list[1])]
no_audio_index = list(no_audio_index.index)

df = df.drop(no_audio_index, axis=0)

In [90]:
df_shuffled = df.sample(frac=1).reset_index(drop=True)
df_shuffled

,file_name,gender,age,record_time,text
0,일반남여_일반통합10_M_1542480084_22_전라_실내_14735.wav,M,22,2.73,하여간 움직이는 거 엄청 싫어해
1,일반남여_일반통합12_M_1523735540_27_수도권_실내_17998.wav,M,27,4.95,참 그러니까 아이들이 활동적으로 놀아야 건강해지는 거겠네
2,일반남여_일반통합05_M_clu5717_33_수도권_실내_07925.wav,M,33,6.16,이 사람은 사이코패스인 사람은 뇌부터 다르다고 생각하는 사람이었어
3,일반남여_일반통합15_F_ioulove486_39_기타_실내_27009.wav,F,39,3.48,그러면 저도 택배 부탁할게요
4,일반남여_일반통합01_M_dongwooo.im_28_기타_실내_00922.wav,M,28,3.92,나도 하나 가입 돼 있어
...,...,...,...,...,...
787551,일반남여_일반통합13_M_1561592011_40_충청_실내_22333.wav,M,40,2.90,이 직장을 계속 다녀야할까
787552,일반남여_일반통합12_M_1579323786_25_수도권_실내_19282.wav,M,25,2.47,아니? 본 적 없어
787553,일반남여_일반통합07_F_1549435934_32_경상_실내_09741.wav,F,32,9.47,사실 무섭기는 하지만 애들의 치과 공포증을 없앨 수만 있다면 꾹 참아봐야지 뭐
787554,일반남여_일반통합07_F_lotusjyh_39_수도권_실내_10574.wav,F,39,5.12,약 이주 치 짓고 기본 검사하고


사람 수 조절

In [91]:
people = 75000 # 사람 수를 조절해서 데이터 길이를 조절합니다.
ftime = df_shuffled[df_shuffled['gender'] == 1][:people]['record_time'].sum() # 여성
mtime = df_shuffled[df_shuffled['gender'] == 0][:people]['record_time'].sum() # 남성

(ftime + mtime) / 3600

199.80121555555553

In [92]:
df_f = df_shuffled[df_shuffled['gender'] == 1][:people]
df_m = df_shuffled[df_shuffled['gender'] == 0][:people]

In [93]:
finaldf = pd.concat([df_f,df_m], axis=0).reset_index(drop=True)
finaldf = finaldf[['file_name','text']]

In [94]:
def modify_file_name(file_name):
    parts = file_name.split('_')
    new_path = f"/DATA2/자유대화 음성(일반남녀)/Training/{'_'.join(parts[:7])}/{file_name}"
    return new_path

# Apply the function to the file_name column
finaldf['file_name'] = finaldf['file_name'].apply(modify_file_name)
finaldf

,file_name,text
0,/DATA2/자유대화 음성(일반남녀)/Training/일반남여_일반통합15_F_io...,그러면 저도 택배 부탁할게요
1,/DATA2/자유대화 음성(일반남녀)/Training/일반남여_일반통합11_F_15...,네 동네에서 신고하신 것 같아요 그런데도 자꾸 오네요
2,/DATA2/자유대화 음성(일반남녀)/Training/일반남여_일반통합11_F_15...,오늘 점심 시간에 공원을 보니까 벌써 주황색 나뭇잎들이 보이더라
3,/DATA2/자유대화 음성(일반남녀)/Training/일반남여_일반통합08_F_15...,또 면역력 증진에 효과적인 음식 중 하나래
4,/DATA2/자유대화 음성(일반남녀)/Training/일반남여_일반통합14_F_15...,신혼부부가 쓸만한 제품을 찾고 있어요
...,...,...
149995,/DATA2/자유대화 음성(일반남녀)/Training/일반남여_일반통합05_M_15...,결국은 뭐 내 말대로네 골고루 먹어라
149996,/DATA2/자유대화 음성(일반남녀)/Training/일반남여_일반통합12_M_15...,알았어 그렇게 할게
149997,/DATA2/자유대화 음성(일반남녀)/Training/일반남여_일반통합09_M_15...,그래서 그때 투자를 더 했어
149998,/DATA2/자유대화 음성(일반남녀)/Training/일반남여_일반통합12_M_15...,그래도 난 그냥 기본적인 우유 식빵을 제일 많이 먹게 되더라


In [95]:
train, valid = train_test_split(finaldf, test_size=splitRate)

train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)

train.to_csv(os.path.join(saveSetPath, 'train.csv'), index=False, header=False)
valid.to_csv(os.path.join(saveSetPath, 'test.csv'), index=False, header=False)